In [1]:
import logging
import os
import platform

import numpy as np
import pybullet as p
import yaml
import matplotlib.pyplot as plt
from PIL import Image

import igibson
from igibson.action_primitives.action_primitive_set_base import ActionPrimitiveError
from igibson.action_primitives.starter_semantic_action_primitives import StarterSemanticActionPrimitives
from igibson.objects.articulated_object import URDFObject
from igibson.robots.behavior_robot import BehaviorRobot
from igibson.robots.fetch import Fetch
from igibson.scenes.igibson_indoor_scene import InteractiveIndoorScene
from igibson.simulator import Simulator
from igibson.utils.assets_utils import get_ig_avg_category_specs, get_ig_model_path
from igibson.utils.utils import parse_config


 _   _____  _  _
(_) / ____|(_)| |
 _ | |  __  _ | |__   ___   ___   _ __
| || | |_ || || '_ \ / __| / _ \ | '_ \
| || |__| || || |_) |\__ \| (_) || | | |
|_| \_____||_||_.__/ |___/ \___/ |_| |_|



In [2]:
def execute_controller(ctrl_gen, robot, s):
    for action in ctrl_gen:
        robot.apply_action(action)
        s.step()


def go_to_table(s, robot, controller: StarterSemanticActionPrimitives):
    """Go to the table object in the scene."""
    for i in range(20):
        try:
            table = s.scene.objects_by_category["coffee_table"][0]
            print("Trying to NAVIGATE_TO table.")
            execute_controller(controller._navigate_to_obj(table), robot, s)
            print("NAVIGATE_TO table succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to navigate to table failed. Retry until 20.".format(i + 1))
            continue

        return

def go_to_table_simpler(s, robot, controller: StarterSemanticActionPrimitives):
    """Go to the table object in the scene."""
    table = s.scene.objects_by_category["coffee_table"][0]
    print("Trying to NAVIGATE_TO table.")
    execute_controller(controller._navigate_to_obj(table), robot, s)
    print("NAVIGATE_TO table succeeded!")
    return

In [3]:
logging.basicConfig(level=logging.INFO)

s = Simulator(
    mode="headless",
    image_width=512,
    image_height=512,
    device_idx=0,
    use_pb_gui=False,
)
scene = InteractiveIndoorScene(
    "Rs_int", load_object_categories=["walls", "floors", "bottom_cabinet", "door", "sink", "coffee_table", "fridge"]
)
s.import_scene(scene)

# Load the robot and place it in the scene.
config = parse_config(os.path.join(igibson.configs_path, "behavior_robot_mp_behavior_task.yaml"))
config["robot"]["show_visual_head"] = True
robot = BehaviorRobot(**config["robot"])
s.import_robot(robot)

robot.set_position_orientation([0, 0, 1], p.getQuaternionFromEuler([0, 0, -np.pi/4])) # positive offset on the third dim -> turn left, negavite -> turn right
robot.apply_action(
    np.zeros(
        robot.action_dim,
    )
)

# Run some steps to let physics settle.
for _ in range(300):
    s.step()

INFO:igibson.render.mesh_renderer.get_available_devices:Device 0 is available for rendering
INFO:igibson.render.mesh_renderer.get_available_devices:Command '['/home/nicola/python_projects/iGibson/igibson/render/mesh_renderer/build/test_device', '1']' returned non-zero exit status 1.
INFO:igibson.render.mesh_renderer.get_available_devices:Device 1 is not available for rendering
INFO:igibson.scenes.scene_base:Loading scene...
INFO:igibson.scenes.scene_base:Scene loaded!


																																										b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
left_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
right_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
neckb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warn

In [4]:
import pyquaternion  

def render_robot_pov(robot, step):
    robot_pos, robot_orientation = robot.get_position_orientation()
    
    # Convert quaternion to rotation matrix - takes w,x,y,z in input, but robot orientation is given as x,y,z,w !!!
    q = pyquaternion.Quaternion(x=robot_orientation[0], 
                                y=robot_orientation[1], 
                                z=robot_orientation[2], 
                                w=robot_orientation[3])
    
    forward_downward_direction = q.rotate(np.array([1, 0, -0.25]))  # Default forward vector (x-axis)
    up_direction = q.rotate(np.array([0, 0, 1]))  # Default up vector (z-axis)
    
    # Set the camera at the robot's head level (optional: raise it slightly)
    camera_pose = robot_pos + q.rotate(np.array([0.1, 0.1, 1])) # Slightly above the robot's center
    
    # Set the camera in the renderer
    s.renderer.set_camera(camera_pose, camera_pose + forward_downward_direction, up_direction)
    frame = s.renderer.render(modes=("rgb"))[0]
    rgb_image = (frame[..., :3] * 255).astype(np.uint8) 
    
    # Ensure the directory exists
    os.makedirs("./images/rendering_attempts", exist_ok=True)
    
    # Save using PIL
    image = Image.fromarray(rgb_image)
    #image.show()
    image.save(f"./images/rendering_attempts/img_1st_person_{step}.jpg", "JPEG")

In [8]:
controller = StarterSemanticActionPrimitives(None, scene, robot)
obj = s.scene.objects_by_category["coffee_table"][0]
ctrl_gen = controller._navigate_to_obj(obj)
for i, action in enumerate(ctrl_gen):
    robot.apply_action(action)
    s.step()
    if i % 5 == 0:
        render_robot_pov(robot, i)
    